<a href="https://colab.research.google.com/github/xiangivyli/data_engineering_zoomcamp_learning/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [4]:
#define the square_root_generator
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

In [5]:
# Question 1: What is the sum of the outputs of the generator for limit = 5?
sum = 0
generator_5 = square_root_generator(5)


for sqrt_value in generator_5:
    sum += sqrt_value
print(sum)

8.382332347441762


In [6]:
#Question 2: What is the 13th number yielded
limit2 = 13
generator_13 = square_root_generator(limit2)

for sqrt_value in generator_13:
  print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


In [7]:
type(generator_13)

generator

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [19]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [7]:
from itertools import chain

In [8]:
# Use itertools.chain() to combine both generators
combined_generator = chain(people_1(), people_2())

# Convert the combined generator into a list
for person in combined_generator:
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [12]:
# generator can only be used once, need to create it again
combined_generator = chain(people_1(), people_2())

sum_age_append = 0

for person in combined_generator:
  sum_age_append += person['Age']

print(sum_age_append)

353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [1]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [2]:
import dlt

In [20]:
#load the people_1 firstly
load_pipeline = dlt.pipeline(pipeline_name="workshop_homework_people",
						destination='duckdb',
						dataset_name='people')

# run the pipeline with default settings, and capture the outcome
info = load_pipeline.run(people_1(),
                    table_name="people_update",
                    write_disposition="replace",
                    primary_key = "id")

# show the outcome
print(info)

Pipeline workshop_homework_people load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/workshop_homework_people.duckdb location to store data
Load package 1708185930.9354627 is LOADED and contains no failed jobs


In [21]:
# show outcome

import duckdb

conn = duckdb.connect(f"{load_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{load_pipeline.dataset_name}'")
print('Loaded people_1 tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_1 table below:")

people_1 = conn.sql("SELECT * FROM people_update").df()
display(people_1)


Loaded people_1 tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_update       │
└─────────────────────┘




 people_1 table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708185930.9354627,3hvCoOQ2d5T9/A
1,2,Person_2,27,City_A,1708185930.9354627,xji6z6vuI0jDHg
2,3,Person_3,28,City_A,1708185930.9354627,HM1/Abz0a3J5VQ
3,4,Person_4,29,City_A,1708185930.9354627,VUJXsboGedgwQg
4,5,Person_5,30,City_A,1708185930.9354627,klHHmhyqXeaEKQ


In [22]:
#merge the people_2 then
load_pipeline

# run the pipeline with default settings, and capture the outcome
info_2 = load_pipeline.run(people_2(),
                    table_name="people_update",
                    write_disposition="merge",
                    primary_key = "id")

# show the outcome
print(info_2)

Pipeline workshop_homework_people load step completed in 0.61 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/workshop_homework_people.duckdb location to store data
Load package 1708186014.872558 is LOADED and contains no failed jobs


In [23]:
conn = duckdb.connect(f"{load_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{load_pipeline.dataset_name}'")
print('Merge 2 tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n merged table below:")

people_merge = conn.sql("SELECT * FROM people_update").df()
display(people_merge)

Merge 2 tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_update       │
└─────────────────────┘




 merged table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708185930.9354627,3hvCoOQ2d5T9/A,None
1,2,Person_2,27,City_A,1708185930.9354627,xji6z6vuI0jDHg,None
2,5,Person_5,35,City_B,1708186014.872558,cE2jw559jyRa8g,Job_5
3,8,Person_8,38,City_B,1708186014.872558,kow81iIZHw/V0Q,Job_8
4,7,Person_7,37,City_B,1708186014.872558,afA7qvqVAI2j2g,Job_7
5,4,Person_4,34,City_B,1708186014.872558,NXPBWS+fWkeAHQ,Job_4
6,3,Person_3,33,City_B,1708186014.872558,XjHL72gv0R465Q,Job_3
7,6,Person_6,36,City_B,1708186014.872558,8UjtEe+jUpejEA,Job_6


In [24]:
# sum all the ages
print("\n\n\n sum of age")

age = conn.sql("""
SELECT sum(age)
FROM people_update
""")
display(age)




 sum of age


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX